In [46]:
import pandas as pd
import os
from datetime import datetime

# Mapping of NBA team IDs to Oddshark IDs
team_to_oddshark_id = {
    "1610612737": "20734", # Atlanta Hawks
    "1610612738": "20722", # Boston Celtics
    "1610612751": "20749", # Brooklyn Nets
    "1610612766": "20751", # Charlotte Hornets
    "1610612741": "20732", # Chicago Bulls
    "1610612739": "20735", # Cleveland Cavaliers
    "1610612742": "20727", # Dallas Mavericks
    "1610612743": "20723", # Denver Nuggets
    "1610612765": "20743", # Detroit Pistons
    "1610612744": "20741", # Golden State Warriors
    "1610612745": "20740", # Houston Rockets
    "1610612754": "20737", # Indiana Pacers
    "1610612746": "20736", # LA Clippers
    "1610612747": "20739", # Los Angeles Lakers
    "1610612763": "20729", # Memphis Grizzlies
    "1610612748": "20726", # Miami Heat
    "1610612749": "20725", # Milwaukee Bucks
    "1610612750": "20744", # Minnesota Timberwolves
    "1610612740": "20733", # New Orleans Pelicans
    "1610612752": "20747", # New York Knicks
    "1610612760": "20728", # Oklahoma City Thunder
    "1610612753": "20750", # Orlando Magic
    "1610612755": "20731", # Philadelphia 76ers
    "1610612756": "20730", # Phoenix Suns
    "1610612757": "20748", # Portland Trail Blazers
    "1610612758": "20745", # Sacramento Kings
    "1610612759": "20724", # San Antonio Spurs
    "1610612761": "20742", # Toronto Raptors
    "1610612762": "20738", # Utah Jazz
    "1610612764": "20746"  # Washington Wizards
}

def fetch_over_under_points(oddshark_id, season_id, game_date):
    year = int(season_id[1:]) + 1
    year = str(year)
    url = f"https://www.oddsshark.com/stats/gamelog/basketball/nba/{oddshark_id}?season={year}"
    
    try:
        tables = pd.read_html(url)
        df = tables[0]
    except Exception as e:
        print(f"BAD - error for team {oddshark_id} on date {game_date}: {e}")
        return None
    
    if df.empty:
        print(f"BAD - No data in table for team {oddshark_id} on date {game_date}")
        return None
    
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
    matching_rows = df[df['Date'] == game_date]
    
    if matching_rows.empty:
        print(f"BAD - No matching date found for team {oddshark_id} on date {game_date}")
        return None
    
    over_under = matching_rows.iloc[0]['Total']
    return over_under

def update_gamelogs_with_over_under(game_pks_file, gamelogs_folder):
    # Load all the games from the file
    game_data = pd.read_csv(game_pks_file, dtype=str)
    game_data = game_data.head(10) # Modify the number here to change the amount of files to update
    
    # Iterate over the games
    for _, row in game_data.iterrows():
        game_id = row['GAME_ID']
        game_date = pd.to_datetime(row['GAME_DATE'])
        home_team_id = row['HOME_ID']
        season_id = row['SEASON_ID']
        
        # Get the corresponding Oddshark team IDs
        home_oddshark_id = team_to_oddshark_id.get(home_team_id)

        # Fetch over/under points for both teams
        home_over_under = fetch_over_under_points(home_oddshark_id, season_id, game_date)

        # Read the game log CSV
        game_file = os.path.join(gamelogs_folder, f'game_{game_id}.csv')
        if os.path.exists(game_file):
            game_df = pd.read_csv(game_file)

            # Add new columns for over/under points
            game_df['Over_Under_Total'] = home_over_under

            # Save the updated game log back to the CSV
            game_df.to_csv(game_file, index=False)
            print(f"Updated {game_file} with over/under points")
        else:
            print(f"Game file {game_file} does not exist.")

# Update the paths as necessary
game_pks_file = 'nba_REGULARSEASON_ONLY_game_pks.csv'
gamelogs_folder = 'nba_gamelogs'

# Run the update function
update_gamelogs_with_over_under(game_pks_file, gamelogs_folder)

Updated nba_gamelogs/game_0022000002.csv with over/under points
Updated nba_gamelogs/game_0022000001.csv with over/under points
Updated nba_gamelogs/game_0022000012.csv with over/under points
Updated nba_gamelogs/game_0022000018.csv with over/under points
Updated nba_gamelogs/game_0022000019.csv with over/under points
Updated nba_gamelogs/game_0022000003.csv with over/under points
Updated nba_gamelogs/game_0022000020.csv with over/under points
Updated nba_gamelogs/game_0022000013.csv with over/under points
Updated nba_gamelogs/game_0022000014.csv with over/under points
Updated nba_gamelogs/game_0022000010.csv with over/under points
